## General Packages

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import xsimlab as xs
import numba #for wrapping the function
import zarr
import matplotlib.pyplot as plt
from scipy.stats import uniform
from scipy.linalg import lstsq
from ipyfastscape import TopoViz3d
#%load_ext xsimlab.ipython
#plt.style.use('default')
from dask.diagnostics import ProgressBar
from scipy.stats import gmean
import sys
import os

## 1) Import the Measured Data

In [2]:
print(os.getcwd())

/Users/amandawild/Desktop/ClimateSignalPropagation_AlluvialFanGrainSize/Part2_GrapevineMountains/SetUp_MisfitInversion


In [3]:
#Run only once
os.chdir('..')
os.chdir('..')
os.chdir('Data_Darcyetal2017')
print(os.getcwd())

/Users/amandawild/Desktop/ClimateSignalPropagation_AlluvialFanGrainSize/Data_Darcyetal2017


In [4]:
# Load data from CSV files related to geomorphological features.
Moonlight = pd.read_csv('Moonlight_holoQ2.csv')
BackThrust = pd.read_csv('Backthrust_holoQ2.csv')
EndofDataSetIndex=18

# Remove NaN values from relevant columns in the datasets.
def drop_na_columns(df):
    return {col: df[col].dropna(inplace=False) for col in ['Distance', 'Elevation', 'Q2Distance', 'Q2Elevation']}

moonlight_data = drop_na_columns(Moonlight)
backthrust_data = drop_na_columns(BackThrust)

# Calculate slopes based on elevation and distance data.
def calculate_slope(elevation, distance):
    return (elevation[0] - elevation[EndofDataSetIndex]) / (distance[EndofDataSetIndex] * 1000)

MoonlightSlope = calculate_slope(moonlight_data['Elevation'], moonlight_data['Distance'])
BackThrustSlope = calculate_slope(backthrust_data['Elevation'], backthrust_data['Distance'])
BackThrustQ2Slope = calculate_slope(backthrust_data['Q2Elevation'], backthrust_data['Q2Distance'])
MoonlightQ2Slope = calculate_slope(moonlight_data['Q2Elevation'], moonlight_data['Q2Distance'])

# Define constants related to geomorphological measurements.
#Taken directly from Darxy et al. 2019 and Frankel et al.2007
#D’Arcy, M., Whittaker, A.C., and Roda-Boluda, D.C., 2017, Measuring alluvial fan sensitivity to past climate changes using a self-similarity approach to grain-size fining, Death Valley, California: Sedimentology, v. 64, no. 1, p. 388–424, https://doi.org/10.1111/sed.12308.
#Frankel, K.L., Brantley, K.S., Dolan, J.F., Finkel, R.C., Klinger, R.E., Knott, J.R., Machette, M.N., Owen, L.A., Phillips, F.M., Slate, J.L., et al., 2007, Cosmogenic 10Be and 36Cl geochronology of offset alluvial fans along the northern Death Valley fault zone: Implications for transient strain in the eastern California shear zone: Journal of Geophysical Research: Solid Earth, v. 112, no. B8, https://doi.org/10.1029/2006JB004350.
ERMeasured = 0.000065
ERerror = 0.00002
MoonLightBaseLevel = 200  # Approximate base level at river
BackthrustBaseLevel = 325
MeanBaselevel = (MoonLightBaseLevel + BackthrustBaseLevel) / 2

# Measured using Darcy et al. 2019 profiles and  Verified in google earth 
#Measured using a freeely available DEM and Verified in google earth
FanMeasured = 723  
FanError = 200  
MeanMountainHeight = 1408  
MaxMountainHeight = 2665  
MaxOrogenError = 400  

AvgApexGS = (BackThrust.MEAND.max() + BackThrust.Q2MEAND.max() + Moonlight.MEAND.max() + Moonlight.Q2MEAND.max()) / 4

# Calculate measured slope error between Q2 and standard slopes
MeasuredSlopeError = np.round(np.sqrt(np.max(np.array([((MoonlightQ2Slope * 100) - (MoonlightSlope * 100))**2,
          ((BackThrustSlope * 100) - (BackThrustQ2Slope * 100))**2,
                                              ((BackThrustSlope * 100) - (MoonlightSlope * 100))**2,
                                              ((BackThrustQ2Slope * 100) - (MoonlightQ2Slope * 100))**2
                                             ]))),1)
#(MoonlightQ2Slope * 100) - (MoonlightSlope * 100)

# Calculate errors based on grain size differences at the apex
GSError = np.round(np.sqrt(np.max([(BackThrust.MEAND[0] - Moonlight.MEAND[0]) ** 2,
                  (BackThrust.Q2MEAND[0] - Moonlight.Q2MEAND[0]) ** 2,
                          (BackThrust.Q2MEAND[0] - BackThrust.MEAND[0]) ** 2,
                          (Moonlight.Q2MEAND[0] - Moonlight.MEAND[0]) ** 2, 
                 ])))

In [5]:
# last_valid_indexB = np.where(~np.isnan(BackThrust.MEAND))[0][-1]
# last_valid_indexBQ = np.where(~np.isnan(BackThrust.Q2MEAND))[0][-1]
# #
# last_valid_indexM = np.where(~np.isnan(Moonlight.MEAND))[0][-1]
# last_valid_indexMQ = np.where(~np.isnan(Moonlight.Q2MEAND))[0][-1]
# GSError2 = np.round(np.sqrt(np.max([(BackThrust.MEAND[last_valid_indexB] - Moonlight.MEAND[last_valid_indexM]) ** 2,
#                   (BackThrust.Q2MEAND[last_valid_indexBQ] - Moonlight.Q2MEAND[last_valid_indexMQ]) ** 2,
#                           (BackThrust.Q2MEAND[last_valid_indexBQ] - BackThrust.MEAND[last_valid_indexB]) ** 2,
#                           (Moonlight.Q2MEAND[last_valid_indexMQ] - Moonlight.MEAND[last_valid_indexM]) ** 2, 
#                  ])))

## 2) Prepare the Death VAlley Fastscape/GravelScape model Set Up

## import model packages

In [6]:
#
os.chdir('..')
os.chdir('Part2_GrapevineMountains/SetUp_MisfitInversion')
print(os.getcwd())

/Users/amandawild/Desktop/ClimateSignalPropagation_AlluvialFanGrainSize/Part2_GrapevineMountains/SetUp_MisfitInversion


In [9]:
#The GravelScape Model

In [29]:
from GravelScape import GravelGSN

In [30]:
#from fastscape.models import basic_model
from fastscape.models import sediment_model
from fastscape.processes import FlowAccumulator
from fastscape.processes import (MultipleFlowRouter)
from fastscape.processes import (Bedrock)
from fastscape.processes import (DrainageArea)
#from fastscape.processes import StratigraphicHorizons
model = sediment_model.update_processes({'flow': MultipleFlowRouter,
                                         'GravelGSN':GravelGSN,
                                         #'strati': StratigraphicHorizons,
                                         'bedrock':Bedrock})

## Set up the spatial and temporal constraints of the model set up

In [31]:
### temporal

In [32]:
tf=17e6 #duration of fault activity 
TimeStep=5000 #yrs
timestepbtw_simulation= 10000 #50000 matched the data best
ntime=int(tf/timestepbtw_simulation)+1
timestepbtw_output=timestepbtw_simulation*5
nout=int(tf/timestepbtw_output)+1
print('Number of time steps:',tf/timestepbtw_simulation)
print('Number of outputs:',tf/timestepbtw_output)

Number of time steps: 1700.0
Number of outputs: 340.0


In [33]:
# Determine Steady State Averaging interval
Steady=tf*0.9
Steadytime=int(((tf-Steady)/(tf/nout-1))+1)
SteadytimeLEN=len(np.linspace(Steady,tf,int(Steadytime)))

In [34]:
### Spatial

In [35]:
#Real World Data
MeanFanorCatchmentWidth=2e3#2.5e3 #meters
NumberofFansorCatchments=2#2.8
wm = 17e3 # width of the mountain (in m)
wb = 7e3 # with of the basin (in m)

CellSize=100 #Width of larger channels #m 
Boundary=['core','fixed_value','looped','looped']

#Convert this to model Inputs
xl = wm+wb # xl, length of the grid in the x-direction 
yl = MeanFanorCatchmentWidth*NumberofFansorCatchments # yl, length of the mesh in the y-direction 
nx=int(xl/CellSize)+1 #(ranges between 2 (over 20 for channel dynamics) to ~1000.)
ny=int(yl/CellSize)+1 #(ranges between 2(over 20 for channel dynamics) to ~1000.)
x, y = np.meshgrid(np.linspace(0,xl,nx), np.linspace(0,yl,ny)) # creates an x,y mesh
Boundary=['core','fixed_value','looped','looped']

print('Min. Number of Cells Across basin, down basin, or down mountain', int(np.min([wb,wm,yl])/(CellSize)))
print('2D Channel dynamics in the basin are possible',np.where(int(np.min([wb,yl])/(CellSize)>20),'True','False'))
print('The grid cells are square',(x[0,0:2])==(y[0:2,0]))

Min. Number of Cells Across basin, down basin, or down mountain 40
2D Channel dynamics in the basin are possible True
The grid cells are square [ True  True]


In [36]:
# Additional processes that can be updated after defining the spatial/temporal constraints

In [37]:
#Reduce Precip
#Oscillate Precip over time (with the option for irragular intervals)
from fastscape.processes import FlowAccumulator
model=model.update_processes({'drainage': FlowAccumulator})

CatchmentPrecip1=1#0.65610394
BasinPrecip=1

time = np.linspace(0, tf, ntime)
pr = xr.DataArray(np.where(
    (x[np.newaxis,:, :]<wm)&
    (time[:,np.newaxis,np.newaxis] <=tf),
    CatchmentPrecip1,BasinPrecip),
                  dims=['time','y','x'])

In [38]:
h0 = np.ones([ny,nx]) 
model=model.drop_processes('init_topography')

In [39]:
# FastScape

In [40]:
def run_fastscape_FinalStep(Kb,Ks,s0,alpha,U,G,h0=h0,tf=tf,ntime=ntime,Steady=Steady,Steadytime=Steadytime,ny=ny,nx=nx,yl=yl,wm=wm,wb=wb,xl=xl,x=x):

    ds_in = xs.create_setup(
        model=model,
        clocks={'time': np.linspace(0,tf,ntime),
               'out': np.linspace(Steady,tf,int(Steadytime))},
        master_clock = 'time',
        input_vars={
            # nb. of grid nodes in (y, x)
            'grid__shape': [ny,nx],
            # total grid length in (y, x)
            'grid__length': [yl,xl],
            # node status at borders
            'boundary__status': ['core','fixed_value','looped','looped'],
            # uplift rate
            'uplift__rate': xr.DataArray(np.where(x<wm, U, -s0*np.exp(-(x-wm)/wb*alpha)),dims=['y','x']), ##xr.DataArray(np.where(s==0, U[np.newaxis,np.newaxis], s[:,:]),dims=['y','x']), #
            # random seed
            #'init_topography__seed': None,
            'topography__elevation': h0*262.5,
            # MFD partioner slope exponent
            'flow__slope_exp': 1.0,
            # drainage area exponent
            'spl__area_exp': 0.4,
            # slope exponent
            'spl__slope_exp': 1,
            # bedrock channel incision coefficient
            'spl__k_coef_bedrock': (Kb),#4e-5,
            # soil (sediment) channel incision coefficient
            'spl__k_coef_soil': Ks,#K_s, #0.5e-5,
            # detached bedrock transport/deposition coefficient
            'spl__g_coef_bedrock': G,#0.8,
            # soil (sediment) transport/deposition coefficient
            'spl__g_coef_soil': G,
            # bedrock diffusivity
            'diffusion__diffusivity_bedrock': 0,#(0.03),
            # soil (sediment) diffusivity
            'diffusion__diffusivity_soil': 0,#0.1,
            # The bedrock source D50 grain size
            'GravelGSN__D0': 1,
            # The bedrock source grain size standard deviation
            'GravelGSN__SD0': 0.8, #18.09/33.3=0.54?? #0.75 default
            # Cv (grain size calculation constant) described in Fedele and Paola (2007) and Duller
            # et al. (2010)
            'GravelGSN__Cv': 0.8, #Darcy
            # C1 (grain size calculation constant) described in Fedele and Paola (2007) and Duller
            # et al. (2010)
            'GravelGSN__C1': 0.7, #Darcy
            # Porosity
            'GravelGSN__porosity': 0.3,
            # runoff
            'drainage__runoff':1 #xr.DataArray(np.where(x<wm,PR,1), dims=['y','x']) ,

        },
        output_vars={'topography__elevation': 'out',
                     'GravelGSN__DMean': 'out',
                     'GravelGSN__DTIME': 'out',
                     'drainage__flowacc': 'out',
                     'erosion__rate': 'out',
                    }
    )
    out_ds = (ds_in.xsimlab.run(model=model))
    return out_ds

## 3) Calculate the Misfit

In [41]:
from mango import scheduler, Tuner
from mango.domain.distribution import loguniform

In [42]:
# Prepare arrays for misfit calculations.
# Parameter space definition for optimization using Mango's Tuner.
#Option to change the range and run the optimizer again to further the analysis and in case of local minina issues. 
param_space = {
    'Kb': loguniform(5e-7, 5e-6), #logunifrom or unifrom
    'Ks': loguniform(5e-7, 5e-6),
    's0': uniform(0.00005, 0.00015),
    'alpha': uniform(0.3,3),
    'U': uniform(0.000135, 0.000155),
    'G': uniform(0.3,3)
}
# Define model indices for fan slope calculations
startfan = 170
endfan = -1

In [43]:
#Prep for the misfit (make arrays)
BackThrustGSIndex=(np.array(round(BackThrust.Distance[:-2]*10)).astype(int))
BackThrustQ2GSIndex=(np.array(round(BackThrust.Q2Distance*10)).astype(int))
MoonlightQ2GSIndex=(np.array(round(Moonlight.Q2Distance*10)).astype(int))
MoonlightGSIndex=(np.array(round(Moonlight.Distance[:-2]*10)).astype(int))
#
MoonlightGSMisfit=np.ones_like(MoonlightGSIndex).astype(float)
MoonlightQ2GSMisfit=np.ones_like(MoonlightQ2GSIndex).astype(float)
BackThrustGSMisfit=np.ones_like(BackThrustGSIndex).astype(float)
BackThrustQ2GSMisfit=np.ones_like(BackThrustQ2GSIndex).astype(float)
#
MeanMoonlightGSMisfit=np.ones_like(MoonlightGSIndex).astype(float)
MeanMoonlightQ2GSMisfit=np.ones_like(MoonlightQ2GSIndex).astype(float)
MeanBackThrustGSMisfit=np.ones_like(BackThrustGSIndex).astype(float)
MeanBackThrustQ2GSMisfit=np.ones_like(BackThrustQ2GSIndex).astype(float)
#
MoonlightTopoMisfit=np.ones_like(MoonlightGSIndex).astype(float)
MoonlightQ2TopoMisfit=np.ones_like(MoonlightQ2GSIndex).astype(float)
BackThrustTopoMisfit=np.ones_like(BackThrustGSIndex).astype(float)
BackThrustQ2TopoMisfit=np.ones_like(BackThrustQ2GSIndex).astype(float)
#

In [44]:
# For more information of the mango inversion method please see: 
#https://github.com/ARM-software/mango (Sandha et al. 2020; 2021)

In [45]:
from mango import scheduler, Tuner
from mango.domain.distribution import loguniform
from scipy.stats import uniform

@scheduler.parallel(n_jobs=-1)
def obj(Kb,Ks,s0,alpha,U,G):
    #Run Fastscape with the new parameters
    out_elevation = run_fastscape_FinalStep(Kb,Ks,s0,alpha,U,G)
    #Analyze the topographic parameters
    Topo=xr.DataArray(out_elevation.topography__elevation.values,dims=['out','y','x'])
    Topo=Topo.mean('y') #final out step
    FanSlope=Topo.isel(x=slice(startfan,endfan)).isel(out=-1).differentiate('x').values
    LowSlope=np.sum(np.where((FanSlope*-1)<0.001,1,0))/71
    #GS=GS.mean('out').mean('y')#average of frinal 10% of simulation
    ModelMountainTopo=(Topo.isel(x=slice(0,startfan-1)).isel(out=-1).mean().values)
    MaxOrogen=(Topo.isel(x=slice(0,startfan-1)).isel(out=-1).max().values)
    FanApex=(Topo.isel(x=slice(startfan,startfan+10)).isel(out=-1).max().values)
    FanMinEl=(Topo.isel(x=slice(startfan,-1)).isel(out=-1).min().values)
    #Analyze the catchment wide erosion rates
    ER=xr.DataArray(out_elevation.erosion__rate.values,dims=['out','y','x'])
    ER2=ER.isel(x=slice(0,startfan-1)).isel(out=-1).mean('y').mean('x').values
    #Analyze the grain size fining in the larger channels and across all channels
    Drainage=xr.DataArray(out_elevation.drainage__flowacc.values,dims=['out','y','x']).isel(x=slice(startfan,endfan)).isel(out=slice(-20,-1))
    GS=xr.DataArray(out_elevation.GravelGSN__DMean.values,dims=['out','y','x']).isel(x=slice(startfan,endfan)).isel(out=slice(-20,-1))
    GSMean=GS.mean('out').mean('y')  
    FinaloutGmeanQ=gmean(Drainage.mean('out').mean('y')) 
    BinaryQ=xr.DataArray(np.where(Drainage>=FinaloutGmeanQ,1,0),dims=['out','y','x'])
    GS_LargeChannels=xr.DataArray(np.where(BinaryQ==1,GS,np.nan),dims=['out','y','x'])
    GS_LargeChannels=GS_LargeChannels.mean('out').mean('y')
    #
    # CALCULATE THE MISFIT
    #Check for local minima and flat topography and assign a high misfit to avoid these simulations 
    if (ModelMountainTopo<1) or (FanApex<1)or (FanMinEl<1):
        TotalMisfit=9999 
        TotalMisfit2=9999
    elif (LowSlope>0.1):
        TotalMisfit=9999
        TotalMisfit2=9999
    else: #Calculate the misfit
        for i in range(0,len(MoonlightGSIndex)):
            MoonlightGSMisfit[i]=((np.array(Moonlight.MEAND[i])-
                           (GS_LargeChannels*AvgApexGS)[MoonlightGSIndex[i]])**2)/GSError
            BackThrustGSMisfit[i]=((np.array(BackThrust.MEAND[i])-
                           (GS_LargeChannels*AvgApexGS)[BackThrustGSIndex[i]])**2)/GSError
            MeanMoonlightGSMisfit[i]=((np.array(Moonlight.MEAND[i])-
                           (GSMean*AvgApexGS)[MoonlightGSIndex[i]])**2)/GSError
            MeanBackThrustGSMisfit[i]=((np.array(BackThrust.MEAND[i])-
                           (GSMean*AvgApexGS)[BackThrustGSIndex[i]])**2)/GSError
            MoonlightTopoMisfit[i]=((Moonlight.Elevation[i]-(Topo.isel(x=slice(startfan,endfan)).isel(out=-1)[MoonlightGSIndex[i]]))**2)/(FanError**2)
            BackThrustTopoMisfit[i]=((BackThrust.Elevation[i]-(Topo.isel(x=slice(startfan,endfan)).isel(out=-1)[BackThrustGSIndex[i]]))**2)/(FanError**2)
            
        for i in range(0,len(MoonlightQ2GSIndex)):
            MoonlightQ2GSMisfit[i]=((np.array(Moonlight.Q2MEAND[i])-
                           (GS_LargeChannels*AvgApexGS)[MoonlightQ2GSIndex[i]])**2)/GSError
            BackThrustQ2GSMisfit[i]=((np.array(BackThrust.Q2MEAND[i])-
                           (GS_LargeChannels*AvgApexGS)[BackThrustQ2GSIndex[i]])**2)/GSError
            MeanMoonlightQ2GSMisfit[i]=((np.array(Moonlight.Q2MEAND[i])-
                           (GSMean*AvgApexGS)[MoonlightQ2GSIndex[i]])**2)/GSError
            MeanBackThrustQ2GSMisfit[i]=((np.array(BackThrust.Q2MEAND[i])-
                           (GSMean*AvgApexGS)[BackThrustQ2GSIndex[i]])**2)/GSError
            MoonlightQ2TopoMisfit[i]=((Moonlight.Q2Elevation[i]-(Topo.isel(x=slice(startfan,endfan)).isel(out=-1)[MoonlightQ2GSIndex[i]]))**2)/(FanError**2)
            BackThrustQ2TopoMisfit[i]=((BackThrust.Q2Elevation[i]-(Topo.isel(x=slice(startfan,endfan)).isel(out=-1)[BackThrustQ2GSIndex[i]]))**2)/(FanError**2)  
        Moonslope=((np.nanmean(FanSlope[1:]*-1)-(MoonlightSlope*100))/MeasuredSlopeError)**2
        MoonslopeQ2=((np.nanmean(FanSlope[1:]*-1)-(MoonlightQ2Slope*100))/MeasuredSlopeError)**2
        BackthrustQ2slope=((np.nanmean(FanSlope[1:]*-1)-(BackThrustQ2Slope*100))/MeasuredSlopeError)**2
        Backthrustslope=((np.nanmean(FanSlope[1:]*-1)-(BackThrustSlope*100))/MeasuredSlopeError)**2
        #
        FanSlopeMisfit=np.nanmean([Moonslope,MoonslopeQ2,BackthrustQ2slope,Backthrustslope])
        ChannelGSFit=np.nanmean([(np.nanmean(BackThrustGSMisfit),np.nanmean(BackThrustQ2GSMisfit),np.nanmean(MoonlightGSMisfit),np.nanmean(MoonlightQ2GSMisfit))])
        MeanGSFit=np.nanmean([(np.nanmean(MeanBackThrustGSMisfit),np.nanmean(MeanBackThrustQ2GSMisfit),np.nanmean(MeanMoonlightGSMisfit),np.nanmean(MeanMoonlightQ2GSMisfit))])
        fanmisfit=np.nanmean([(np.nanmean(BackThrustTopoMisfit),np.nanmean(BackThrustQ2TopoMisfit),np.nanmean(MoonlightTopoMisfit),np.nanmean(MoonlightQ2TopoMisfit))])
        #((FanApex-FanMeasured)/FanError)**2
        ERMisfit=((ER2-ERMeasured)/ERerror)**2
        OTopoMisfit=((MaxOrogen-MaxMountainHeight)/MeanBaselevel)**2
        #
        #THE FINAL MISFIT
        TotalMisfit=(ChannelGSFit+fanmisfit+ERMisfit+OTopoMisfit)/4
        TotalMisfit2=(ChannelGSFit+fanmisfit+ERMisfit+OTopoMisfit+FanSlopeMisfit+MeanGSFit)/6
        #
        # Save the datasets
        print("Writing ChannelGSMisfits.txt...")
        with open('ChannelGSMisfit.txt', 'a') as f:
            print(str(ChannelGSFit), file=f)
        f.close()
        #
        with open('GMisfit.txt', 'a') as f:
            print(str(G), file=f)
        f.close()
        #
        with open('MeanGSMisfit.txt', 'a') as f:
            print(str(MeanGSFit), file=f)
        f.close()
        #
        with open('fanElevationmisfit.txt', 'a') as f:
            print(str(fanmisfit), file=f)
        f.close()
        with open('FanSlopeMisfit.txt', 'a') as f:
            print(str(FanSlopeMisfit), file=f)
        f.close()
        with open('ERMisfit.txt', 'a') as f:
            print(str(ERMisfit), file=f)
        f.close()
        with open('OTopoMisfit.txt', 'a') as f:
            print(str(OTopoMisfit), file=f)
        f.close()
        with open('TotalMisfit.txt', 'a') as f:
            print(str(TotalMisfit), file=f)
        f.close()
        with open('TotalMisfit2.txt', 'a') as f:
            print(str(TotalMisfit2), file=f)
        f.close()
        with open('Kbs.txt', 'a') as f:
            print(str(Kb), file=f)
        f.close()
        with open('Kss.txt', 'a') as f:
            print(str(Ks), file=f)
        f.close()
        with open('Us.txt', 'a') as f:
            print(str(U), file=f)
        f.close()
        with open('S0s.txt', 'a') as f:
            print(str(s0), file=f)
        f.close()
        with open('Alphas.txt', 'a') as f:
            print(str(alpha), file=f)
        f.close()
    return TotalMisfit #TotalMisfit2

  

In [46]:
misfit=Tuner(param_space, obj)
results = misfit.minimize()  

/opt/anaconda3/envs/FastScapeEnvironment/lib/python3.10/site-packages/xsimlab/xr_accessor.py:229: FutureWarning: variable 'time' with name matching its dimension will not be automatically converted into an `IndexVariable` object in the future.
  xr_var = as_variable(data, name=dim)
/opt/anaconda3/envs/FastScapeEnvironment/lib/python3.10/site-packages/xsimlab/xr_accessor.py:229: FutureWarning: variable 'time' with name matching its dimension will not be automatically converted into an `IndexVariable` object in the future.
  xr_var = as_variable(data, name=dim)
/opt/anaconda3/envs/FastScapeEnvironment/lib/python3.10/site-packages/xsimlab/xr_accessor.py:229: FutureWarning: variable 'out' with name matching its dimension will not be automatically converted into an `IndexVariable` object in the future.
  xr_var = as_variable(data, name=dim)
/opt/anaconda3/envs/FastScapeEnvironment/lib/python3.10/site-packages/xsimlab/xr_accessor.py:229: FutureWarning: variable 'out' with name matching its d

  0%|          | 0/20 [00:00<?, ?it/s]

/opt/anaconda3/envs/FastScapeEnvironment/lib/python3.10/site-packages/xsimlab/xr_accessor.py:229: FutureWarning: variable 'time' with name matching its dimension will not be automatically converted into an `IndexVariable` object in the future.
  xr_var = as_variable(data, name=dim)
/opt/anaconda3/envs/FastScapeEnvironment/lib/python3.10/site-packages/xsimlab/xr_accessor.py:229: FutureWarning: variable 'out' with name matching its dimension will not be automatically converted into an `IndexVariable` object in the future.
  xr_var = as_variable(data, name=dim)
/opt/anaconda3/envs/FastScapeEnvironment/lib/python3.10/site-packages/xsimlab/xr_accessor.py:229: FutureWarning: variable 'time' with name matching its dimension will not be automatically converted into an `IndexVariable` object in the future.
  xr_var = as_variable(data, name=dim)
/opt/anaconda3/envs/FastScapeEnvironment/lib/python3.10/site-packages/xsimlab/xr_accessor.py:229: FutureWarning: variable 'out' with name matching its d

KeyboardInterrupt: 

In [27]:
# Save dictionary
np.save('March2024_OrogenBasinGSMisfit.npy', results) 

NameError: name 'results' is not defined